In [1]:
import time
import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import datasetPlt as dataset
import cv2
from sklearn.metrics import confusion_matrix
from datetime import timedelta
%matplotlib inline
correct_sum = 0

In [18]:
# Convolutional Layer 1.
filter_size1 = 5
num_filters1 = 16

# Convolutional Layer 2.
filter_size2 = 3
num_filters2 = 32

# Convolutional Layer 3.
filter_size3 = 3
num_filters3 = 32

# Convolutional Layer 4.
filter_size3 = 4
num_filters3 = 8

# Fully-connected layer.
fc_size = 128            

num_channels = 3
img_size = 64

img_size_flat = img_size * img_size * num_channels

img_shape = ( img_size, img_size)


classes = ['Apple_Apple_scab',
           'Apple_Black_rot','Apple_healthy','Apple_Cedar_apple_rust','Blueberry_healthy','Cherry_healthy',
           'Cherry_Powdery_mildew','Corn_Cercospora_leaf_spot Gray_leaf_spot','Corn_Common_rust','Corn_Northern_Leaf_Blight',
           'Corn_healthy','Grape_Black_rot','Grape_Esca_(Black_Measles)','Grape_healthy','Grape_Leaf_blight',
           'Orange_Haunglongbing','Peach_Bacterial_spot','Pepper_bell_Bacterial_spot','Pepper_bell_healthy',
           'Peach_healthy','Potato_Early_blight','Potato_healthy','Potato_Late_blight','Raspberry_healthy',
           'Soybean_healthy','Squash_Powdery_mildew','Strawberry_healthy','Strawberry_Leaf_scorch',
           'Tomato_Bacterial_spot','Tomato_Tomato_Yellow_Leaf_Curl_Virus','Tomato_Tomato_mosaic_virus','Tomato_Target_Spot',
           'Tomato_Early_blight','Tomato_Spider_mites_Two-spotted_spider_mite','Tomato_Septoria_leaf_spot',
           'Tomato_Leaf_Mold','Tomato_Late_blight','Tomato_healthy']

num_classes = len(classes)

batch_size = 32


validation_size = .16


early_stopping = None

train_path = 'C:/Users/Saurabh/Desktop/Train'
test_path = 'plnt_villa/test'
print("done")

done


In [3]:
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)


#cv2.imshow('img', test_images[0])
#cv2.waitKey(500)

38
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Reading training images
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
7589


In [4]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))


Size of:
- Training-set:		6375


In [5]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [6]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [7]:
def new_conv_layer(input,              
                   num_input_channels, 
                   filter_size,        
                   num_filters,        
                   use_pooling=True):

   
    shape = [filter_size, filter_size, num_input_channels, num_filters]

   
    weights = new_weights(shape=shape)

   
    biases = new_biases(length=num_filters)
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    layer += biases
    
    

    if use_pooling:
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')

    layer = tf.nn.relu(layer)
    
    
    


    return layer, weights

In [8]:
def flatten_layer(layer): 

    layer_shape = layer.get_shape()

    num_features = layer_shape[1:4].num_elements()
    
    layer_flat = tf.reshape(layer, [-1, num_features])

    return layer_flat, num_features

In [9]:
def new_fc_layer(input,          
                 num_inputs,     
                 num_outputs,    
                 use_relu=True,
                 drop_out = True): 

    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    layer = tf.matmul(input, weights) + biases
    if drop_out:
        layer = tf.nn.dropout(layer , 0.60)

    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [10]:
x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')
x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [11]:
layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1,
                   use_pooling=True)
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 32, 32, 16) dtype=float32>

In [12]:
layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2,
                   use_pooling=True)
layer_conv2

<tf.Tensor 'Relu_1:0' shape=(?, 16, 16, 32) dtype=float32>

In [13]:
layer_conv3, weights_conv3 = \
    new_conv_layer(input=layer_conv2,
                   num_input_channels=num_filters2,
                   filter_size=filter_size3,
                   num_filters=num_filters3,
                   use_pooling=False)
layer_conv3

<tf.Tensor 'Relu_2:0' shape=(?, 16, 16, 8) dtype=float32>

In [14]:
layer_flat, num_features = flatten_layer(layer_conv3)
layer_flat
num_features

2048

In [15]:
layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True,
                         drop_out = True)
layer_fc1

<tf.Tensor 'Relu_3:0' shape=(?, 128) dtype=float32>

In [16]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=num_classes,
                         use_relu=False,
                         drop_out = False)
layer_fc2

<tf.Tensor 'add_4:0' shape=(?, 38) dtype=float32>

In [17]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, dimension=1)
y_pred
y_pred_cls

Instructions for updating:
Use the `axis` argument instead


<tf.Tensor 'ArgMax_1:0' shape=(?,) dtype=int64>

In [19]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,labels=y_true)
cost = tf.reduce_mean(cross_entropy)

In [20]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-5).minimize(cost)

In [21]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
session = tf.Session()
#session run
session.run(tf.initialize_all_variables())
saver = tf.train.Saver()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [23]:
train_batch_size = batch_size

In [24]:
def print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss, train_loss):
 

    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Epoch {0} - T_A: {1:>6.1%}, V_A: {2:>6.1%}, Validation Loss: {3:.3f}, train_loss : {4:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss, train_loss))
    return epoch + 1, acc, val_acc, val_loss

In [25]:

total_iterations = 0


def optimize(num_iterations):

    
    global total_iterations
    # time-usage
    start_time = time.time()
    
    best_val_loss = float("inf")
    patience = 0

    for i in range(total_iterations, total_iterations + num_iterations):

        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)

    
        x_batch = x_batch.reshape(train_batch_size, img_size_flat)
        x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        feed_dict_validate = {x: x_valid_batch,
                              y_true: y_valid_batch}

    
        #print( "{}  \100 ".format(i) ,end='\r')
        session.run(optimizer, feed_dict=feed_dict_train)

        if i%5 == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_validate)
            train_loss = session.run(cost, feed_dict=feed_dict_train)
            
            epoch = total_iterations
          
            epoch, ta, va, vl = print_progress(epoch, feed_dict_train, feed_dict_validate, val_loss, train_loss)
            
            
            

            
            if early_stopping:    
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience = 0
                else:
                    patience += 1

                if patience == early_stopping:
                    break
            total_iterations += 1

    end_time = time.time()

    time_dif = end_time - start_time

    print("Time elapsed: " + str(timedelta(seconds=int(round(time_dif)))))

In [26]:

def print_test_accuracy():
    
    num_test = len(data.valid.images)

    correct_sum = 0
    
    cls_pred = np.zeros(shape=num_test, dtype=np.int)


    i = 0

    while i < num_test:
        j = min(i + batch_size, num_test)
        
        images = data.valid.images[i:j, :].reshape(j-i, img_size_flat)
        
       # print(j-i)
    
        labels = data.valid.labels[i:j, :]

        feed_dict = {x: images,
                     y_true: labels}
        
        
        
        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j
        
    cls_true = np.array(data.valid.cls)
    cls_pred = np.array([classes[x] for x in cls_pred]) 

    
    correct = (cls_true == cls_pred)

  
    correct_sum = correct.sum()

    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))

In [27]:
print ( "start")
optimize(num_iterations=1) 

saver.restore(session,'model5.ckpt')

start
Epoch 1 - T_A:   3.1%, V_A:   0.0%, Validation Loss: 3.639, train_loss : 3.631
Time elapsed: 0:00:07
INFO:tensorflow:Restoring parameters from model5.ckpt


In [28]:
print_test_accuracy()
optimize(num_iterations=20)


Accuracy on Test-Set: 61.4% (745 / 1214)
Epoch 2 - T_A:  59.4%, V_A:  50.0%, Validation Loss: 1.566, train_loss : 1.592
Epoch 3 - T_A:  68.8%, V_A:  81.2%, Validation Loss: 1.005, train_loss : 0.911
Epoch 4 - T_A:  62.5%, V_A:  75.0%, Validation Loss: 1.272, train_loss : 1.170
Epoch 5 - T_A:  71.9%, V_A:  75.0%, Validation Loss: 0.962, train_loss : 1.266
Time elapsed: 0:00:01


In [29]:
print_test_accuracy( )

Accuracy on Test-Set: 61.0% (741 / 1214)


In [30]:

img_path = 'C:/Users/Saurabh/Desktop/Train/Cherry_healthy/CH (55).jpg'

img = cv2.imread(img_path)
img = cv2.resize(img, (64,64), cv2.INTER_LINEAR)

img = img.astype('float32')
img = img / 255

img = img.reshape( -1 , img_size_flat)


feed_dict = {x: img}
cls_pred = session.run(y_pred_cls, feed_dict = feed_dict)

In [31]:
print( classes[int(cls_pred)] )

Cherry_healthy
